In [4]:
import shutil
import subprocess
import os
from itertools import groupby
from operator import itemgetter
import numpy as np
import csv
import os
import time
import copy
import pandas as pd

os.chdir(os.path.expanduser('~/Dropbox/smacpy'))

from smacpy import Smacpy


In [ ]:
#############################
## Enter Speaker Name Here ##
#############################

speaker = "Baldwin, James"

corpus_dir = "/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/"

#############################

last_name = speaker.split(', ')[0]

print(speaker)
print(last_name)

In [16]:

ubm_dir = os.path.join(corpus_dir,'UBM_'+last_name)
speaker_dir =os.path.join(corpus_dir,speaker)

num_speaker_clips=len([item for item in os.listdir(speaker_dir) if item.lower()[-4:]=='.wav'])

smacpy_dict={}

for filename in [item for item in os.listdir(ubm_dir) if '.wav' in item]:
	smacpy_dict[os.path.join(ubm_dir,filename)]="background"

for filename in [item for item in os.listdir(speaker_dir) if '.wav' in item]:
	smacpy_dict[os.path.join(speaker_dir,filename)]=speaker_dir.strip('/').split('/')[-1]

model = Smacpy("", smacpy_dict)

In [5]:

def smooth(x,window_len=10,window='hanning'):
        if x.ndim != 1:
                raise ValueError, "smooth only accepts 1 dimension arrays."
        if x.size < window_len:
                raise ValueError, "Input vector needs to be bigger than window size."
        if window_len<3:
                return x
        if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
                raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
        s=np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
        if window == 'flat': #moving average
                w=np.ones(window_len,'d')
        else:  
                w=eval('np.'+window+'(window_len)')
        y=np.convolve(w/w.sum(),s,mode='same')
        return y[window_len:-window_len+1]





def seconds_list_to_ranges(seconds_list): 
    ranges = []                
    for k, g in groupby(enumerate(seconds_list), lambda (i,x):i-x):
        group = map(itemgetter(1), g)
        ranges.append((group[0], group[-1]))
    return ranges

seconds_list_to_ranges([1,2,3,7,8,9,34,99,100,101,102,199])

In [17]:

def classify_audio_file(audio_pathname):
    segments_dir_temp = os.path.join(corpus_dir, last_name+'_segments_1_sec')
    basename = audio_pathname.split('/')[-1][:-4]
    try:
        os.mkdir(segments_dir_temp)
        subprocess.call(['ffmpeg','-i',audio_pathname,'-n','-f','segment','-segment_time','1','-c','copy',os.path.join(segments_dir_temp,basename+"_sec_%05d.wav")])
    except:
        print("Apparently already processed: "+audio_pathname)
    seg_pathnames = [os.path.join(segments_dir_temp, item) for item in os.listdir(segments_dir_temp) if item.lower()[-4:]=='.wav']
    output=[]
    global speaker
    for filename in [item for item in seg_pathnames if '.wav' in item]:
        if model.classify(filename) == speaker:
            output.append(1)
        else:
            output.append(0)
    
    counter=0
    speaker_secs=[]
    for segment in output:
        if segment>0.0:
            speaker_secs.append(counter)
        counter+=1
    speaker_ranges = seconds_list_to_ranges(speaker_secs)
    csv_pathname = audio_pathname[:-4]+'_'+last_name+'_GMM_UBM_'+str(num_speaker_clips)+"x2s.csv"
    with open(csv_pathname, 'w') as csv_fo:
        speaker_ranges_expanded=[(start,1,end-start+1) for start,end in speaker_ranges]
        csv_writer = csv.writer(csv_fo)
        csv_writer.writerows(speaker_ranges_expanded)
    ## Smooth version
    output_smooth_temp = list(smooth(np.array(output)))
    output_smooth=[]
    for item in output_smooth_temp:
        output_smooth.append(round(item))
    counter=0
    speaker_secs=[]
    for segment in output_smooth:
        if segment>0.0:
            speaker_secs.append(counter)
        counter+=1
    speaker_ranges = seconds_list_to_ranges(speaker_secs)
    csv_smooth_pathname = audio_pathname[:-4]+'_'+last_name+'_GMM_UBM_smooth10_'+str(num_speaker_clips)+"x2s.csv"
    with open(csv_smooth_pathname, 'w') as csv_fo:
        speaker_ranges_expanded=[(start,1,end-start+1) for start,end in speaker_ranges]
        csv_writer = csv.writer(csv_fo)
        csv_writer.writerows(speaker_ranges_expanded)
    shutil.rmtree(segments_dir_temp)


    


In [ ]:
to_classify=["/Volumes/U/AAPB_Corpus_May_2017/Test_Sets/AAPB_Test_Haystack_Baldwin/cpb-aacip-15-0v89g5gf5r__barcode94696_clip1_.h264.16000.wav",'/Volumes/U/AAPB_Corpus_May_2017/Test_Sets/AAPB_Test_Haystack_Baldwin/cpb-aacip-15-83xsjk31.16000.wav', '/Volumes/U/AAPB_Corpus_May_2017/Test_Sets/AAPB_Test_Haystack_Baldwin/cpb-aacip-28-8s4jm23q52__PRA_AAPP_BB0632_A_conversation_with_James_Baldwin_.16000.wav', '/Volumes/U/AAPB_Corpus_May_2017/Test_Sets/AAPB_Test_Haystack_Baldwin/cpb-aacip-28-9z90863j5g__PRA_AAPP_BB2011_Free_and_brave_.16000.wav', '/Volumes/U/AAPB_Corpus_May_2017/Test_Sets/AAPB_Test_Haystack_Baldwin/cpb-aacip-500-2v2ccw7g.16000.wav', '/Volumes/U/AAPB_Corpus_May_2017/Test_Sets/AAPB_Test_Haystack_Baldwin/cpb-aacip-500-mc8rgt5t.16000.wav']

for pathname in to_classify:
    classify_audio_file(pathname)